# Project: Car Detection and Tracking using OpenCV

This project implements a simple vehicle detection and tracking system using Python and OpenCV.  
It uses background subtraction and a distance-based tracking algorithm to identify and follow moving cars in a video feed.

-- 
## 🔍 Steps Breakdown

### 1. **Video Input & ROI Selection**
Capture frames from a traffic video using OpenCV and define a **Region of Interest (ROI)** — the part of the frame where vehicles are expected.  
This reduces noise and focuses processing power on the relevant area.

---

### 2. **Background Subtraction**
Apply `cv2.createBackgroundSubtractorMOG2()` to distinguish moving objects from the static background.  
This isolates cars and other vehicles from the road when the camera is stable.

---

### 3. **Thresholding**
Convert the background mask into a binary image using `cv2.threshold()` —  
white pixels represent detected motion, and black pixels represent static background.

---

### 4. **Mask Cleaning (Morphological Operations)**
Apply operations like **erosion and dilation** (e.g., `cv2.morphologyEx`) to:
- Remove small noisy areas
- Fill holes in detected motion blobs
- Improve contour detection

---

### 5. **Finding Contours**
Detect **contours** (object outlines) in the thresholded mask using `cv2.findContours()`.  
Each contour corresponds to a potential vehicle.

---

### 6. **Filtering Small Contours**
Ignore small contours that are unlikely to be vehicles based on their **area size**.  
This reduces false positives caused by shadows, small debris, or distant objects.

---

### 7. **Bounding Boxes & Detection List**
For each valid contour:
- Calculate its bounding rectangle using `cv2.boundingRect()`
- Add it to a **detections list** for tracking

---

### 8. **Object Tracking with Euclidean Distance**
Use a custom `EuclideanDistTracker` class to:
- Assign a unique **ID** to each detected vehicle
- Match vehicles across frames based on proximity
- Maintain consistent IDs as cars move

---

### 9. **Drawing Rectangles with IDs**
For each tracked vehicle:
- Draw a **rectangle** around it
- Display the **ID** using `cv2.putText()` for visual feedback

---

### 10. **Saving Output Video (Optional)**
Write each processed frame to a new video file using `cv2.VideoWriter()`  
This creates an output with sual tracking results for further analysis or presentation.

---

## ✅ Outcome

- Tracks moving cars in a video stream
- Assigns consistent IDs to each vehicle
- Visualizes bounding boxes motion in real time
- Optionally exports results as a video file

---

## 🛠️ Possible Extensions

- Vehicle counting (entry/exit zones)
- Speed estimation
- License plate detection
- Deep learning-based object detection (e.g., YOLO, SSD)
- GUI dashboard for live monitoring and analytics

---


In [5]:
# main.py

import cv2
import os
from tracker import EuclideanDistTracker

# Set working directory if needed
# os.chdir("D:/object tracking/object_tracking")

# Create tracker object
tracker = EuclideanDistTracker()

# Load video
cap = cv2.VideoCapture("highway.mp4")

# Get frame dimensions
ret, frame = cap.read()
if not ret:
    print("Error loading video.")
    exit()

frame_height, frame_width, _ = frame.shape

# Save output video with proper dimensions
result = cv2.VideoWriter('output_tracking.mp4', 
                         cv2.VideoWriter_fourcc(*'XVID'),
                         20, (frame_width, frame_height))

# Background subtractor
object_detector = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=50)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Region of Interest (ROI) - adjust as needed
    roi = frame[340: 720, 500: 800]

    # 1. Object Detection
    mask = object_detector.apply(roi)
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)

    # Optional: Clean mask
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)


    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    detections = []

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 100:
            x, y, w, h = cv2.boundingRect(cnt)
            detections.append([x, y, w, h])

    # 2. Object Tracking
    boxes_ids = tracker.update(detections)
    for box_id in boxes_ids:
        x, y, w, h, id = box_id
        cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(roi, f'ID {id}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    # Show frames
    frame[340: 720, 500: 800] = roi  # Update the main frame with ROI overlay
    cv2.imshow("ROI", roi)
    cv2.imshow("Mask", mask)
    cv2.imshow("Frame", frame)

    result.write(frame)

    key = cv2.waitKey(30)
    if key == 27:
        break

# Cleanup
cap.release()
result.release()
cv2.destroyAllWindows()


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\morph.dispatch.cpp:1163: error: (-215:Assertion failed) !_src.empty() in function 'cv::morphologyEx'
